# 名刺管理ツール

## 概要
- PDFスキャンした名刺を、OCRとLLMを使用し、Google Spread Sheetsに整理します。
- 名刺の画像は'card_images', 名刺のOCR情報は'personal_info.gsheet'に保存されます。

## 実行方法
1. ScanSnap等で名刺を両面印刷したPDFを'input_pdfs'にアップロードしてください。
1. 上記タブの"ランタイム/ランタイムのタイプを変更"からT4 GPUもしくはTPU v2に変更してください。
1. 上記タブの"ランタイム/全て実行"をクリックしてください。

## 備考
- Google Vision APIとChatGPTを使用したほうが、精度はいいです。
- このプログラムは無料で使用できるように、paddle OCRとllama3を使用しています。

© 2024 Hiroki Nakanishi


In [8]:
# IMPORT
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

from google.colab import drive
drive.mount('/content/drive')

!pip install blinker --ignore-installed
!pip uninstall -y Pillow --ignore-installed
!apt-get install poppler-utils
!curl https://ollama.ai/install.sh | sh
!echo 'debconf debconf/frontend select Noninteractive' | sudo debconf-set-selections
!sudo apt-get update && sudo apt-get install -y cuda-drivers
import os
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

!nohup ollama serve &
!pip install pypdf2 pdf2image
!pip install paddlepaddle paddleocr
!ollama pull llama3
!ollama pull phi3
!pip install ollama

from paddleocr import PaddleOCR
import ollama
import PyPDF2
from pdf2image import convert_from_path
import io
import os
import glob
import pandas as pd
import json
import numpy as np
from tqdm.notebook import tqdm
!pip install openai
from openai import OpenAI
import warnings
import logging

!sudo apt-get install poppler-utils
from google.colab import files
!pip install kora
from kora.xattr import get_id

import uuid
import shutil
from datetime import datetime

import time
!sudo apt-get install pciutils
!pip install git+https://github.com/HawkClaws/oyama.git ollama
from oyama import oyama

# FOLDER
base_path = '/content/drive/MyDrive/mymy/Businesscard'
input_folder = f'{base_path}/input_pdfs'
tmp_folder = f'{base_path}/input_pdfs/tmp_pdf'
os.makedirs(tmp_folder, exist_ok=True)
save_folder = f'{base_path}/card_images/'
os.makedirs(save_folder, exist_ok=True)

# LLM MODEL
# model_name = 'phi3'
model_name = 'llama3'
# model_name = oyama.run("https://huggingface.co/mmnga/ELYZA-japanese-Llama-2-7b-instruct-gguf/resolve/main/ELYZA-japanese-Llama-2-7b-instruct-q4_K_M.gguf?download=true")

# SPREAD SHEET URL
ss_url = "https://docs.google.com/spreadsheets/d/1a6kBfooirvxrDV1YryjUW7TNYBq824s105dPAlH8yy0/edit#gid=1780343650"

# SPLIT PDF
input_pdfs = glob.glob(f'{input_folder}/*.pdf')

def split_pdf(input_pdf, output_prefix):
    with open(input_pdf, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        num_pages = len(reader.pages)

        for i in range(0, num_pages, 2):
            writer = PyPDF2.PdfWriter()
            for j in range(i, min(i + 2, num_pages)):
                writer.add_page(reader.pages[j])

            output_file = f"{output_prefix}_part_{i // 2 + 1}.pdf"
            with open(output_file, 'wb') as out:
                writer.write(out)

for input_pdf in input_pdfs:
    base_name = os.path.splitext(os.path.basename(input_pdf))[0]
    output_prefix = f"{tmp_folder}/biz_{base_name}_split"
    split_pdf(input_pdf, output_prefix)


# OCR and LLM EXTRACT
data = []
src_files = glob.glob(f'{tmp_folder}/*.pdf')
src_files.sort()
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',    # required but ignored
)


for file_name in tqdm(src_files):

    # OCR TWO SIDES OF BUISINESSCARD
    images = convert_from_path(file_name)
    string = ''
    for i, image in enumerate(images):
          gray_image = image.convert('L')
          numpy_array = np.array(gray_image)

          ocr = PaddleOCR(
            use_gpu=True, #if you have GPU
            lang = "japan", #OCR target language
            det_limit_side_len=gray_image.size[1],
            show_log=False,
            use_angle_cls=True
          )
          result = ocr.ocr(numpy_array, cls=True)
          if result is not None and isinstance(result, list):
              for line in result:
                  if isinstance(line, list):
                      string += ' '.join([word_info[1][0] for word_info in line if word_info is not None and len(word_info) > 1]) + '\n'

    # LLM EXTRACT
    schema = {
        "会社名": "string",
        "部署名": "string",
        "氏名": "string",
        "氏名（英語）": "string",
        "会社住所": "string | null",
        "電話番号": "string | null",
        "e-mailアドレス": "string | null",
    }

    response = client.chat.completions.create(
      # messages=[
      #   {"role": "system", "content": f"あなたは優秀な日本語が使えるアシスタントです。次の文字列から日本語が含まれている場合のみ、会社名、部署名、氏名、会社住所、電話番号、e-mailアドレスを抜き出して、JSON形式で出力してください。英語の情報は無視してください。JSONのスキーマは次の通りです : {schema}"},        {"role": "user", "content": string}
      # ],
      messages=[
        {"role": "system", "content": f"あなたは優秀な日本語が使えるアシスタントです。次の文字列からなるべく日本語表記の会社名、日本語表記の部署名、日本語表記の氏名、英語表記の氏名、日本語表記の会社住所、電話番号、e-mailアドレスを抜き出して、JSON形式で出力してください。複数の経歴がある場合は代表的なものを会社名, 役職名として。JSONのスキーマは次の通りです : {schema}"},
        {"role": "user", "content": string}
      ],
      model=model_name,
      response_format={ "type": "json_object" }
    )
    personal_info = response.choices[0].message.content

    # SAVE PDF
    file_id = str(uuid.uuid4())  # Unique image ID
    pdf_path  = os.path.join(save_folder, f"{file_id}.pdf")
    with open(pdf_path, 'wb') as pdf_output:
        with open(file_name, 'rb') as pdf_input:
            pdf_output.write(pdf_input.read())

    # ADD EXTRA INFO
    data_row = {}
    data_row["追加日"] = datetime.now().strftime('%Y-%m-%d')
    data_row["名刺画像"] = pdf_path
    data_row.update(json.loads(personal_info))

    # ADD DATA to TABLE
    data.append(data_row)

# COMPLETED TABLE
df = pd.DataFrame(data).iloc[:, 0:9] # extract data culomn
display(df)

# WAIT FOR PDF SAVED
time.sleep(60)

# GET DRIVE URL
def get_drive_url(file_path):
    return f'https://drive.google.com/file/d/{get_id(file_path)}/view'

df['名刺画像'] = df['名刺画像'].apply(get_drive_url)


# ADD TABLE TO SPREADSHEET
workbook = gc.open_by_url(ss_url)
worksheet = workbook.get_worksheet(0)
df.fillna('', inplace=True)
df.replace({None: ''}, inplace=True)
df_list = df.values.tolist()
headers = df.columns.tolist()
df_list.insert(0, headers)
existing_data = worksheet.get_all_values()
if len(existing_data) > 0:
    df_list = df_list[1:]
worksheet.append_rows(df_list)

# CLEAN UP
if os.path.exists(tmp_folder):
    shutil.rmtree(tmp_folder)
for pdf in input_pdfs:
    os.remove(pdf)

# DONE
display("Done!")



KeyboardInterrupt: 